In [1]:
from openvino.runtime import Core
ie = Core()
print("使用Core()初始化推理引擎: {}".format(ie))

print("显示主机CPU/GPU设备信息。")
devices = ie.available_devices
for device in devices:
    device_name = ie.get_property(device_name=device, property="FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")


使用Core()初始化推理引擎: <Core: available plugins[CPU, GPU.0, GPU.1]>
显示主机CPU/GPU设备信息。
CPU: 13th Gen Intel(R) Core(TM) i9-13900HX
GPU.0: Intel(R) RaptorLake-S Mobile Graphics Controller (iGPU)
GPU.1: NVIDIA GeForce RTX 4060 Laptop GPU (dGPU)


In [2]:
from openvino.runtime import Core, Model

ie = Core()
classification_model_xml = r"bicycle_weights/best_openvino_model/best.xml"

model_ = ie.read_model(model=classification_model_xml)
model = ie.compile_model(model=model_, device_name="CPU")


In [ ]:
# model = core.compile_model("bicycle_weights/best_openvino_model/best.xml", "AUTO")
# model_helmet = core.compile_model("runs/detect/train7/weights/best_openvino_model/best.xml", "AUTO")

In [3]:
model_xml = r"runs/detect/train7/weights/best_openvino_model/best.xml"

model_h = ie.read_model(model=model_xml)
model_helmet = ie.compile_model(model=model_xml, device_name="CPU")

In [4]:
ir = model.create_infer_request()

In [15]:
# 定义预处理函数
def preprocess(image):
    # Preprocess image data from OpenCV
    [height, width, _] = image.shape
    # length = max((height, width))
    # letter_box = np.zeros((length, length, 3), np.uint8)
    # letter_box[0:height, 0:width] = image
    blob = cv2.dnn.blobFromImage(image, scalefactor=1 / 255, size=(width, height), swapRB=True)
    return blob

In [ ]:
# infer_request = model.create_infer_request()
# infer_request_helmet = model_helmet.create_infer_request()

In [5]:
from openvino.runtime import Core
import cv2
import numpy as np
input_layer = model.input(0)
output_layer = model.output(0)
print("- model input: {}".format(input_layer))
print("- model output: {}".format(output_layer))


- model input: <ConstOutput: names[x] shape[?,3,?,?] type: f32>
- model output: <ConstOutput: names[] shape[?,10,21..] type: f32>


In [16]:
image_filename = "test_image_1.jpg"
image = cv2.imread(image_filename)
print("- input image shape: {}".format(image.shape))

- input image shape: (720, 1280, 3)


In [17]:
# 根据模型的期望输入尺寸调整图像尺寸
# image = image.transpose((2, 0, 1))  # 从 HWC 转为 CHW
# input_height, input_width = image.shape[1], image.shape[2]
# print(input_height, input_width)
# image = image.reshape(1, 3, input_height, input_width)
# print(image.shape)
# input_data = np.expand_dims(np.transpose(image, (2, 0, 1)), 0).astype(np.float32)
input_data = preprocess(image)
print(input_data.shape)

(1, 3, 720, 1280)


In [21]:
result = ir.infer(input_data)[model.outputs[0]]
print("推理结果:", result)

RuntimeError: Exception from src\inference\src\cpp\infer_request.cpp:223:
Exception from src\plugins\intel_cpu\src\node.cpp:1620:
Shape inference of Concat node with name __module.model.11/aten::cat/Concat failed: Check 'TRShape::merge_into(output_shape, in_copy)' failed at src\core\shape_inference\include\concat_shape_inference.hpp:49:
While validating node 'opset1::Concat __module.model.11/aten::cat/Concat (opset4::Interpolate __module.model.10/aten::upsample_nearest2d/Interpolate[0]:f32[?,256,2..,2..], cpu_plugin_opset::SwishCPU __module.model.22.cv3.2.1.act/aten::silu_/Swish_19[0]:f32[?,128,1..,1..]) -> (f32[?,384,2..,2..])' with friendly_name '__module.model.11/aten::cat/Concat':
Shape inference input shapes {{1,256,46,80},{1,128,45,80}}
Argument shapes are inconsistent; they must have the same rank, and must have equal dimension everywhere except on the concatenation axis (axis 1).


